In [1]:
import pandas as pd

train = pd.read_csv('https://raw.githubusercontent.com/lovedlim/inf/refs/heads/main/p2/ch2/train.csv')
test = pd.read_csv('https://raw.githubusercontent.com/lovedlim/inf/refs/heads/main/p2/ch2/test.csv')

train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29304 entries, 0 to 29303
Data columns (total 16 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   id              29304 non-null  int64  
 1   age             29292 non-null  float64
 2   workclass       27642 non-null  object 
 3   fnlwgt          29304 non-null  int64  
 4   education       29304 non-null  object 
 5   education.num   29304 non-null  int64  
 6   marital.status  29304 non-null  object 
 7   occupation      27636 non-null  object 
 8   relationship    29304 non-null  object 
 9   race            29304 non-null  object 
 10  sex             29304 non-null  object 
 11  capital.gain    29304 non-null  int64  
 12  capital.loss    29304 non-null  int64  
 13  hours.per.week  29291 non-null  float64
 14  native.country  28767 non-null  object 
 15  income          29304 non-null  object 
dtypes: float64(2), int64(5), object(9)
memory usage: 3.6+ MB


In [2]:
y_train = train['income'].replace('>50K', 1).replace('<=50K', 0)
y_train.shape

C:\Users\user\AppData\Local\Temp\ipykernel_10500\2162192979.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  y_train = train['income'].replace('>50K', 1).replace('<=50K', 0)


(29304,)

In [3]:
test.head()

,id,age,workclass,fnlwgt,education,education.num,marital.status,occupation,relationship,race,sex,capital.gain,capital.loss,hours.per.week,native.country
0,11574,39.0,State-gov,114055,Bachelors,13,Never-married,Exec-managerial,Not-in-family,White,Female,0,0,40.0,United-States
1,15847,38.0,Private,254114,Some-college,10,Married-spouse-absent,Prof-specialty,Own-child,Black,Female,0,0,40.0,United-States
2,17655,44.0,State-gov,55395,HS-grad,9,Never-married,Craft-repair,Not-in-family,White,Male,0,0,NaN,United-States
3,19790,47.0,Private,28035,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,50.0,United-States
4,31812,62.0,NaN,186611,HS-grad,9,Never-married,NaN,Not-in-family,White,Male,0,0,40.0,United-States


In [4]:
train.pop('income')

test.head()

,id,age,workclass,fnlwgt,education,education.num,marital.status,occupation,relationship,race,sex,capital.gain,capital.loss,hours.per.week,native.country
0,11574,39.0,State-gov,114055,Bachelors,13,Never-married,Exec-managerial,Not-in-family,White,Female,0,0,40.0,United-States
1,15847,38.0,Private,254114,Some-college,10,Married-spouse-absent,Prof-specialty,Own-child,Black,Female,0,0,40.0,United-States
2,17655,44.0,State-gov,55395,HS-grad,9,Never-married,Craft-repair,Not-in-family,White,Male,0,0,NaN,United-States
3,19790,47.0,Private,28035,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,50.0,United-States
4,31812,62.0,NaN,186611,HS-grad,9,Never-married,NaN,Not-in-family,White,Male,0,0,40.0,United-States


In [5]:
total_data = pd.concat([train, test], axis=0)
total_data.shape

data_oh = pd.get_dummies(total_data)

train = data_oh.iloc[:len(train)]
test = data_oh.iloc[len(train):]

In [6]:
train = pd.get_dummies(train)
train

test = pd.get_dummies(test)

In [7]:
from sklearn.model_selection import train_test_split

X_tr, X_val, y_tr, y_val = train_test_split(train,
                                            y_train,
                                            test_size=0.2,
                                            random_state=0)

print(X_tr.shape, X_val.shape, y_tr.shape, y_val.shape)

(23443, 106) (5861, 106) (23443,) (5861,)


## RandomForestClassifier

In [8]:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(random_state=0)
rf.fit(X_tr, y_tr)

,n_estimators,100
,criterion,'gini'
,max_depth,None
,min_samples_split,2
,min_samples_leaf,1
,min_weight_fraction_leaf,0.0
,max_features,'sqrt'
,max_leaf_nodes,None
,min_impurity_decrease,0.0
,bootstrap,True
,oob_score,False


In [9]:
pred = rf.predict_proba(X_val)
pred[:10]

array([[0.48, 0.52],
       [0.99, 0.01],
       [0.69, 0.31],
       [0.76, 0.24],
       [0.39, 0.61],
       [0.82, 0.18],
       [0.99, 0.01],
       [0.57, 0.43],
       [0.97, 0.03],
       [0.06, 0.94]])

In [10]:
rf.classes_

array([0, 1])

In [11]:
from sklearn.metrics import roc_auc_score

roc_auc_score(y_val, pred[:,1])

0.9109517138113743

In [12]:
pred = rf.predict(X_val)
pred[: 10]

array([1, 0, 0, 0, 1, 0, 0, 0, 0, 1])

In [13]:
from sklearn.metrics import accuracy_score

accuracy_score(y_val, pred)

0.8592390377068759

In [14]:
from sklearn.metrics import f1_score
f1_score(y_val, pred)

0.6742992499013029

## LGBMClassifier

In [15]:
import lightgbm as lgb

lgbmc = lgb.LGBMClassifier(random_state=0, verbose=-1) # verbose = -1 : 표준출력(로그 끄기) 

lgbmc.fit(X_tr, y_tr)

,boosting_type,'gbdt'
,num_leaves,31
,max_depth,-1
,learning_rate,0.1
,n_estimators,100
,subsample_for_bin,200000
,objective,None
,class_weight,None
,min_split_gain,0.0
,min_child_weight,0.001
,min_child_samples,20


In [16]:
pred = lgbmc.predict_proba(X_val)

In [17]:
roc_auc_score(y_val, pred[:, 1])

0.9294357409461188

In [18]:
pred = lgbmc.predict(X_val)
accuracy_score(y_val, pred)

0.8718648694761986

In [19]:
f1_score(y_val, pred)

0.7092528068137824

In [20]:
test = pd.get_dummies(test)

pred = lgbmc.predict_proba(test)
pred

array([[0.91028088, 0.08971912],
       [0.97793392, 0.02206608],
       [0.97426828, 0.02573172],
       ...,
       [0.93289413, 0.06710587],
       [0.99140099, 0.00859901],
       [0.98702641, 0.01297359]], shape=(3257, 2))

In [21]:
pred = lgbmc.predict_proba(test)
pred

array([[0.91028088, 0.08971912],
       [0.97793392, 0.02206608],
       [0.97426828, 0.02573172],
       ...,
       [0.93289413, 0.06710587],
       [0.99140099, 0.00859901],
       [0.98702641, 0.01297359]], shape=(3257, 2))

In [22]:
lgbmc.classes_

array([0, 1])

In [ ]:
# csv 결과 생성 코드(암기)
submit = pd.DataFrame({'pred' : pred[:, 1]})
submit.to_csv('result.csv', index=False) # index 안 지우면 감점

In [ ]:
pd.read_csv('result.csv') # 제출 내용

,pred
0,0.089719
1,0.022066
2,0.025732
3,0.863427
4,0.024870
...,...
3252,0.004416
3253,0.343750
3254,0.067106
3255,0.008599
